In [1]:
import importlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pickle
from pprint import pprint
import random
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets,models, transforms
from torch.utils import data
import random

In [2]:
sys.path.append("/home/kwk/share/mizuho/u00257/vae_clustering/src/")
sys.path.append("/home/kwk/share/mizuho/u00257/vae_clustering/src/lib/")

In [3]:
# import cnn_dataset
# importlib.reload(cnn_dataset)
from cnn_dataset import cnn_dataset
from skimage import io, transform

## num_classesを変更してください！

In [1]:
batch_size = 50

data_pkls = "../data_pkls/"
pkl_files = [data_pkls + "imgs128_1.pkl", data_pkls + "imgs128_2.pkl", data_pkls + "imgs128_3.pkl"]
num_clusters = 1000 ### num_classes 要変更!!
pkl_labels = data_pkls + "labels" + str(num_clusters) +  "_convvae" + ".pkl"
dataset = cnn_dataset(pkl_files, pkl_labels, classes=num_clusters) 

train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

NameError: name 'cnn_dataset' is not defined

In [5]:
img, label = dataset[0]
print(img, img.shape)
print(label)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0') torch.Size([3, 224, 224])
621


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
def train(net, criterion, optimizer, n_epoch = 2, model_name = "cnn"):

    net.train()  # ネットワークを訓練状態へ切り替える
    train_loss = []
    test_loss = []
    loss_interval = 2000 // 50
    for epoch in range(n_epoch):  # 訓練データを複数回(n_epoch 周分)学習する
        t1 = time.time()
        for i, data in enumerate(trainloader, 0):
            # ローダからデータを読み込む; データは [inputs, labels] の形で取得される
            inputs, labels = data[0].to(device), data[1].to(device)

            # 勾配を0に初期化する(逆伝播に備える)
            optimizer.zero_grad()

            # 順伝播 + 逆伝播 + 最適化
            outputs = net(inputs)
            loss = criterion(outputs, labels.to(torch.long))
            loss.backward()
            optimizer.step()

            # 統計を計算する
            if i % loss_interval == (loss_interval - 1):    # loss_interval ミニバッチ毎に計算する
                train_loss.append(loss.item())

                # テストデータに対する損失を計算する(訓練はしない)
                with torch.no_grad():  #勾配計算をしない宣言(逆伝播用の計算グラフを作成しないことでメモリ節約、速度向上する)
                    data = iter(testloader).next()  #  テストデータを1ミニバッチ取得する
                    inputs, labels = data[0].to(device), data[1].to(device)
                    outputs = net(inputs)
                    loss = criterion(outputs, labels.to(torch.long))
                    test_loss.append(loss.item())
        t2 = time.time()
        print('epoch {}/{} finished, loss: {}, time: {}s'.format(epoch+1,n_epoch, train_loss[-1], t2 - t1))
        torch.save(net.state_dict(), "../model_pths/" + model_name + ".pth")
            
    print('Finished Training')
    return train_loss, test_loss

# 損失の変遷を表示する関数
def show_loss(train_loss, test_loss):
    plt.xlabel("iter")
    plt.ylabel("loss")
    x = [i*loss_interval for i in range(len(train_loss))]
    plt.plot(x, train_loss, label='train_loss')
    plt.plot(x, test_loss, label='test_loss')
    plt.legend()
    plt.show()

# ネットワークの予測精度を計算する関数
def check_accuracy(net):
    net.eval()  # ネットワークを評価状態へ切り替える
    ret = []
    with torch.no_grad():  #勾配計算をしない宣言(逆伝播用の計算グラフを作成しないことでメモリ節約、速度向上する)
        for loader, name in [[trainloader, 'train'], [testloader, 'test']]:
            correct = 0
            total = 0
            for data in loader:
                images, labels = data[0].to(device), data[1]
                labels = labels.to(device=device, dtype=torch.int64)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            ret.append(100 * correct / total)
            print('Accuracy of the network on the {} images: {:.2f} %'.format(name, ret[-1]))

    return ret

In [8]:
net = models.vgg16()
net.classifier[6] = nn.Linear(in_features=4096, out_features=num_clusters)
net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## エポック数を変更してください！！

In [9]:
model_name = "cnn1000classes"

In [10]:
n_epoch = 50

#損失関数と optimizer の定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_loss, test_loss = train(net, criterion, optimizer, n_epoch = n_epoch, model_name=model_name)  # ネットワークを訓練する
show_loss(train_loss, test_loss) # 損失の変遷を表示する
acc = check_accuracy(net) # ネットワークの予測精度を計算する関数

epoch 1/50 finished, loss: 6.822393894195557, time: 213.33466339111328s
epoch 2/50 finished, loss: 6.823824405670166, time: 212.28481483459473s
epoch 3/50 finished, loss: 6.902531623840332, time: 212.19919610023499s
epoch 4/50 finished, loss: 6.893877029418945, time: 212.55189776420593s
epoch 5/50 finished, loss: 6.923060894012451, time: 212.45243191719055s
epoch 6/50 finished, loss: 6.835181713104248, time: 212.18410539627075s
epoch 7/50 finished, loss: 6.7937493324279785, time: 212.19052910804749s
epoch 8/50 finished, loss: 6.823001861572266, time: 212.4788212776184s
epoch 9/50 finished, loss: 6.834223747253418, time: 212.1166250705719s
epoch 10/50 finished, loss: 6.85548210144043, time: 212.07891464233398s
epoch 11/50 finished, loss: 6.829977512359619, time: 212.58235669136047s
epoch 12/50 finished, loss: 6.900691032409668, time: 212.43163514137268s
epoch 13/50 finished, loss: 6.829103469848633, time: 211.9722399711609s
epoch 14/50 finished, loss: 6.818103790283203, time: 212.088327

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/u00257/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-e53e60db18ae>", line 7, in <module>
    train_loss, test_loss = train(net, criterion, optimizer, n_epoch = n_epoch, model_name=model_name)  # ネットワークを訓練する
  File "<ipython-input-7-42ed0c5f5d43>", line 9, in train
    for i, data in enumerate(trainloader, 0):
  File "/home/u00257/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 345, in __next__
    data = self._next_data()
  File "/home/u00257/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 385, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/home/u00257/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
 

KeyboardInterrupt: 